In [ ]:
import json
import os
from pathlib import Path
from datetime import datetime
import pandas as pd

In [ ]:
files = os.listdir('json')
files

In [ ]:
leisi = files[-3]
leisi

In [ ]:
with open(os.path.join('json', leisi), mode="r",encoding="utf-8") as f:
    js = json.load(f)
len(js)

In [ ]:
chunk = js[0]
chunk.keys()

In [ ]:
type(chunk['data'])

In [ ]:
chunk['data'].keys()

In [ ]:
len(chunk['data']['text'])

In [ ]:
sentences = chunk['data']['sentences']
type(sentences), len(sentences)

In [ ]:
sentences[5]

In [ ]:
ner = []
for sent in sentences:
    if sent.get('ner'):
        ner.append(sent.get('ner'))
len(ner)

In [ ]:
ner[-10:]

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe

In [ ]:
rows = []
for i, sentence in enumerate(sentences):
    for token in sentence.get('tokens'):
        token.setdefault('sent_ndx', i)
        del token['features']
        rows.append(token)
len(rows)

In [ ]:
def procSentences(sentences, rows):
    for i, sentence in enumerate(sentences):
        for token in sentence.get('tokens'):
            token.setdefault('sent_ndx', i)
            if 'features' in token:
                del token['features']
            rows.append(token)
    return rows

In [ ]:
r = []
r = procSentences(sentences,r)
len(r)

In [ ]:
def procChunk(chunk, row):
    return procSentences(chunk['data']['sentences'], row)

In [ ]:
r = []
r = procChunk(chunk, r)
len(r)

In [ ]:
def procChunks(chunks):
    rows = []
    for chunk in chunks:
        rows = procChunk(chunk,rows)
    return rows

In [ ]:
rs = procChunks(js)
len(rs)

In [ ]:
def getDF(chunks):
    rows = procChunks(chunks)
    return pd.DataFrame(rows)

In [ ]:
def tstamp():
    return datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")


def time_decor(fun):
    def wrap(*args, **kwargs):
        print(f"Before: {fun.__name__} - {tstamp()}")
        res = fun(*args, **kwargs)
        print(f"After: {fun.__name__} - {tstamp()}")
        return res
    return wrap

In [ ]:
@time_decor
def jsonToParq(fname, inpath='json', outpath='parquet', csvpath='csv'):
    with open(os.path.join(inpath, fname), mode="r",encoding="utf-8") as f:
        jschunks = json.load(f)
    df = getDF(jschunks)
    fpath = Path(fname)
    df.to_csv(os.path.join(csvpath, fpath.with_suffix('.csv')))
    df.to_parquet(os.path.join(outpath, fpath.with_suffix('.parquet')))


In [ ]:
def procJsonFolder(folder='json'):
    files = os.listdir(folder)
    for file in files:
        jsonToParq(file, folder)

In [ ]:
procJsonFolder()

In [ ]:
leisi

In [ ]:
d = jsonToParq(leisi)

In [ ]:
jdf = getDF(js)
jdf.head()

In [ ]:
jdf.tail()

In [ ]:
r[:5]

In [ ]:
df = pd.DataFrame(rows)
df.head(30)

In [ ]:
# TODO go through all the chunks

In [ ]:
df.head()

In [ ]:
lpath = Path(leisi)
lpath

In [ ]:
lpath.with_suffix("")

In [ ]:
df.to_csv(lpath.with_suffix('.csv'))

In [ ]:
df.to_parquet(lpath.with_suffix('.parquet'))

In [ ]:
d = pd.read_parquet(lpath.with_suffix('.parquet'))
d.head()